In [33]:
# in order to install the required packages for the snn conversion, 
# you need the following packages
# furthemore you must have installed miniconda or anaconda
# and activated a virtual environment to execute the following commands

# %conda install akida
# %conda install cnn2snn
# %conda install akida-models

In [34]:
import tensorflow as tf
import utils

In [35]:
ds_train, ds_train_info = utils.getDataset('train')
ds_test, ds_test_info = utils.getDataset('test')
ds_eval, ds_eval_info = utils.getDataset('validation')





 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 






 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 






 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 




In [36]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

In [37]:
model_file = get_file(
    "akidanet_imagenet_224_alpha_50.h5",
    "http://data.brainchip.com/models/akidanet/akidanet_imagenet_224_alpha_50.h5",
    cache_subdir='models/akidanet_imagenet')

model = Sequential()
model.add(load_model(model_file)) 
model.add(Dense(4, activation='relu'))

In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"), 
              metrics=['accuracy'])
model.get_layer(index=-1)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./imagenet_models",
                                                 save_weights_only=True,
                                                 verbose=1)

In [39]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [40]:
results = model.evaluate(
  ds_eval
)

print("test loss, test acc:", results)

model.save("models/imagenet_model")

In [42]:
import cv2
import numpy as np

model = tf.keras.models.load_model("models/imagenet_model")

In [53]:
from IPython.display import Image
import matplotlib.pyplot as plt

for ele in ds_eval.take(1):

  predictions = tf.convert_to_tensor([np.array([bbox]) for bbox in model.predict(ele[0])])

  display(predictions)
  display(ele[1])

  images = tf.image.draw_bounding_boxes(
    ele[0], ele[1], [(0, 0, 255) for _ in range(len(ele[0]))], name=None
    )
  
  images = tf.image.draw_bounding_boxes(
    images, predictions, [(0, 255, 0) for _ in range(len(images))], name=None
    )

  for image in images:

    image = (np.array(image)*255).astype(np.uint8)

    from PIL import Image
    im = Image.fromarray(image)
    im.save("tensor.jpeg")

    break

4/4 [==============================] - 1s 201ms/step


<tf.Tensor: shape=(128, 1, 4), dtype=float32, numpy=
array([[[0.20383526, 0.3732101 , 0.39107692, 0.41539916]],

       [[0.20383525, 0.37321   , 0.3910768 , 0.41539904]],

       [[0.20383534, 0.37321037, 0.39107722, 0.41539955]],

       [[0.2038353 , 0.3732102 , 0.39107698, 0.41539928]],

       [[0.20383534, 0.37321037, 0.39107722, 0.41539955]],

       [[0.20383534, 0.37321034, 0.39107716, 0.41539952]],

       [[0.20383538, 0.37321055, 0.3910774 , 0.4153998 ]],

       [[0.20383525, 0.37321004, 0.3910768 , 0.41539907]],

       [[0.20383532, 0.3732103 , 0.39107716, 0.41539946]],

       [[0.20383534, 0.37321037, 0.39107722, 0.41539955]],

       [[0.2038353 , 0.37321022, 0.39107704, 0.41539934]],

       [[0.2038353 , 0.37321028, 0.3910771 , 0.4153994 ]],

       [[0.20383535, 0.37321043, 0.39107728, 0.4153996 ]],

       [[0.20383531, 0.37321028, 0.3910771 , 0.4153994 ]],

       [[0.20383531, 0.37321034, 0.39107716, 0.41539952]],

       [[0.20383535, 0.37321046, 0.39107728, 0.

<tf.Tensor: shape=(128, 1, 4), dtype=float32, numpy=
array([[[0.14975041, 0.4375    , 0.33943427, 0.5175781 ]],

       [[0.23096447, 0.32617188, 0.7385787 , 0.6191406 ]],

       [[0.24745269, 0.39648438, 0.41339156, 0.48046875]],

       [[0.6734993 , 0.47460938, 0.74963397, 0.5625    ]],

       [[0.3375    , 0.40039062, 0.384375  , 0.43945312]],

       [[0.07377049, 0.28710938, 0.9508197 , 0.7324219 ]],

       [[0.4513889 , 0.15820312, 0.8298611 , 0.46875   ]],

       [[0.56764704, 0.27929688, 0.7470588 , 0.34375   ]],

       [[0.09773463, 0.40429688, 0.26796117, 0.5810547 ]],

       [[0.24633431, 0.390625  , 0.47800586, 0.50390625]],

       [[0.18122555, 0.20214844, 0.63428944, 0.65234375]],

       [[0.17774668, 0.26660156, 0.63680893, 0.7519531 ]],

       [[0.00387597, 0.6308594 , 0.14664082, 0.79296875]],

       [[0.05647059, 0.57128906, 0.26764706, 0.7402344 ]],

       [[0.5625    , 0.7421875 , 0.67663044, 0.8203125 ]],

       [[0.09375   , 0.30859375, 0.31835938, 0.

In [ ]:

for box in [bbox, prediction]:

  x = box[0]
  y = box[1]
  xx = box[2]
  yy = box[3]

  print("x ", x, "y ", y, "xx ", xx, "yy", yy)

  cv2.rectangle(image, (x, y), (xx, yy), (0, 0, 255), 2)
  print("x,y,w,h:",x,y,xx,yy)
  
# save resulting image
cv2.imwrite('example.jpg', image)      

# show thresh and result    
cv2.imshow("bounding_box", image)
cv2.waitKey(0)
cv2.destroyAllWindows()